# Исследование объявлений о продаже квартир

### 0.1 Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from geopy.distance import geodesic as gd
from geopy.geocoders import Nominatim as nm
import time
data = pd.read_csv('real_estate_data.csv', sep='\t')

print('Информация о датафрейме:')
data.shape

Информация о датафрейме:


(23699, 22)

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

In [3]:
display(data.head(30))

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16.0,51.00,8,NaN,...,25.00,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,...,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5.0,34.30,4,NaN,...,8.30,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14.0,32.00,13,NaN,...,41.00,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,...,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,...,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,...,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,...,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9.0,43.60,7,NaN,...,6.50,2.0,Санкт-Петербург,50898.0,15008.0,0.0,NaN,0.0,NaN,289.0


In [4]:
data.columns

Index(['total_images', 'last_price', 'total_area', 'first_day_exposition',
       'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor',
       'is_apartment', 'studio', 'open_plan', 'kitchen_area', 'balcony',
       'locality_name', 'airports_nearest', 'cityCenters_nearest',
       'parks_around3000', 'parks_nearest', 'ponds_around3000',
       'ponds_nearest', 'days_exposition'],
      dtype='object')

In [5]:
data.columns[int(len(data.columns) / 2)]

'open_plan'

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Судя на третьей строчке проектной работы по установлению параметров и проведению предобработки и исследовательского анализа данных с архивными объявлениями о продаже квартир в г. Санкт-Петербург: мы обнаруживаем, что вместо представления данных в столбцах: 'studio' (наличие квартиры-студии) и 'open_plan' (наличие свободной планировки) обнаруживание пропуск данных в виде многоточия ('...') и неполную картину над параметрами всех объявлений о продаже квартир в г. Санкт-Петербург<br>
<br>
Поэтому разделим датафрейм data на две части с одинаковым количество строк, где первая часть заканчивается, а вторая часть начинается с одного и того же столбца, а именно с 'open_plan' (из пятой строки), чтобы не запутаться.<br>
<br>
Затем создадим два списка для разделенения всех названий столбцов в данной таблице (first_part и second_part) и цикл, по которому итерация помещает одно названия столбца в один список, а другое название - в другой список и выводим по частям два разделённых датафрейма: first_part_data и second_part_data для полного прочтения всех значений датафрейма data.<br>
<br>
Ещё в второй и четвёртой строчке данной проектной работы, обнаруживаем неэтическое для программистов название столбца: 'cityCenters_nearest', поэтому сразу меняем это название на 'city_сenters_nearest'.<br>
</div>

In [6]:
#data[['cityCenters_nearest']] = data[['city_centers_nearest']]

In [7]:
first_part = []
second_part = []

for i in range(len(data.columns)):
    if(i <= len(data.columns) / 2 - 1):
        first_part.append(data.columns[i])
    elif(i == len(data.columns) / 2):
        first_part.append(data.columns[i])
        second_part.append(data.columns[i])
    else:
        second_part.append(data.columns[i])

In [8]:
first_part_data = data.loc[:, first_part]
display(first_part_data.head(30))

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16.0,51.00,8,NaN,False,False
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,False,False
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5.0,34.30,4,NaN,False,False
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,False,False
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14.0,32.00,13,NaN,False,False
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,False,False
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,False,False
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,False,False
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,False,False
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9.0,43.60,7,NaN,False,False


In [9]:
second_part_data = data.loc[:, second_part]
display(second_part_data.head(30))

,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,False,25.00,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,False,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,False,8.30,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,False,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,False,41.00,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0
5,False,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,False,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,False,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,False,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0
9,False,6.50,2.0,Санкт-Петербург,50898.0,15008.0,0.0,NaN,0.0,NaN,289.0


 ### 0.2 Шаг 2.1 Предобработка данных

<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Судя на третьей строчке проектной работы по установлению параметров и проведению предобработки и исследовательского анализа данных с архивными объявлениями о продаже квартир в г. Санкт-Петербург: мы обнаруживаем, что вместо представления данных в столбцах: 'studio' (наличие квартиры-студии) и 'open_plan' (наличие свободной планировки) обнаруживание пропуск данных в виде многоточия ('...') и неполную картину над параметрами всех объявлений о продаже квартир в г. Санкт-Петербург<br>
<br>
Поэтому разделим датафрейм data на две части с одинаковым количество строк, где первая часть заканчивается, а вторая часть начинается с одного и того же столбца, а именно с 'open_plan' (из пятой строки).<br>
<br>
Затем создадим два списка для разделенения всех названий столбцов в данной таблице (first_part и second_part) и цикл, по которому итерация помещает одно названия столбца в один список, а другое название - в другой список и выводим по частям два разделённых датафрейма: first_part_data и second_part_data для полного прочтения всех значений датафрейма data.<br>
<br>
Ещё в второй и четвёртой строчке данной проектной работы, обнаруживаем неэтическое для программистов название столбца: 'cityCenters_nearest', поэтому сразу меняем это название на 'city_сenters_nearest'.<br>
</div>

In [10]:
data.isna().sum()

total_images                0
last_price                  0
total_area                  0
first_day_exposition        0
rooms                       0
ceiling_height           9195
floors_total               86
living_area              1903
floor                       0
is_apartment            20924
studio                      0
open_plan                   0
kitchen_area             2278
balcony                 11519
locality_name              49
airports_nearest         5542
cityCenters_nearest      5519
parks_around3000         5518
parks_nearest           15620
ponds_around3000         5518
ponds_nearest           14589
days_exposition          3181
dtype: int64

In [11]:
null_columns = []

for value in data.columns: 
    if data[value].count() != data.shape[0]:
        null_columns.append(value)
        
null_columns

['ceiling_height',
 'floors_total',
 'living_area',
 'is_apartment',
 'kitchen_area',
 'balcony',
 'locality_name',
 'airports_nearest',
 'cityCenters_nearest',
 'parks_around3000',
 'parks_nearest',
 'ponds_around3000',
 'ponds_nearest',
 'days_exposition']

In [12]:
for item in null_columns:
    print(f"\n Строки датафрейма, где у столбца '{item}' обнаружены пропущенные значения:")
    display(first_part_data[data[item].isna()].head())
    display(second_part_data[data[item].isna()].head())


 Строки датафрейма, где у столбца 'ceiling_height' обнаружены пропущенные значения:


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,False,False
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,False,False
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,False,False
5,10,2890000.0,30.4,2018-09-10T00:00:00,1,NaN,12.0,14.4,5,NaN,False,False
6,6,3700000.0,37.3,2017-11-02T00:00:00,1,NaN,26.0,10.6,6,NaN,False,False


,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
1,False,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,False,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,False,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
5,False,9.1,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,False,14.4,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0



 Строки датафрейма, где у столбца 'floors_total' обнаружены пропущенные значения:


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan
186,12,11640000.0,65.2,2018-10-02T00:00:00,2,NaN,NaN,30.80,4,NaN,False,False
237,4,2438033.0,28.1,2016-11-23T00:00:00,1,NaN,NaN,20.75,1,NaN,False,False
457,4,9788348.0,70.8,2015-08-01T00:00:00,2,NaN,NaN,38.40,12,NaN,False,False
671,4,6051191.0,93.6,2017-04-06T00:00:00,3,NaN,NaN,47.10,8,NaN,False,False
1757,5,3600000.0,39.0,2017-04-22T00:00:00,1,NaN,NaN,NaN,9,NaN,False,False


...

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11.0,18.60,1,NaN,False,False
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12.0,14.40,5,NaN,False,False
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26.0,10.60,6,NaN,False,False
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,False,False
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27.0,15.43,26,NaN,False,False


,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
1,False,11.00,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
5,False,9.10,NaN,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55.0
6,False,14.40,1.0,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155.0
7,False,18.90,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
8,False,8.81,NaN,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189.0



 Строки датафрейма, где у столбца 'days_exposition' обнаружены пропущенные значения:


,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,False,False
7,5,7915000.0,71.6,2019-04-18T00:00:00,2,NaN,24.0,NaN,22,NaN,False,False
44,13,5350000.0,40.0,2018-11-18T00:00:00,1,NaN,22.0,NaN,3,NaN,False,False
45,17,5200000.0,50.6,2018-12-02T00:00:00,2,2.65,9.0,30.3,7,NaN,False,False
46,17,6600000.0,52.1,2019-01-31T00:00:00,2,2.60,24.0,29.7,9,NaN,False,False


,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,False,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
7,False,18.9,2.0,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,NaN
44,False,NaN,1.0,Санкт-Петербург,30471.0,11603.0,1.0,620.0,1.0,1152.0,NaN
45,False,7.0,NaN,Санкт-Петербург,30011.0,12872.0,0.0,NaN,0.0,NaN,NaN
46,False,8.3,2.0,Санкт-Петербург,15114.0,12702.0,0.0,NaN,0.0,NaN,NaN


<div class="alert alert-info">
<b>Комментарий студента:</b>
<br>
Теперь нужно заполнить пропущенные значения, но, к счастью, среди некоторых данных столбцов есть такие данные, которые можно логически определить чисто из здравого смысла, реже с помощью условий из других значений столбцов.<br>  
<br> Ими относятся: 
    <ul> 'is_apartment' (наличие аппартаментов); </ul>
    <ul> 'balcony' (наличие балконов); </ul>
    <ul> 'locality_name' (название населённого пункта); </ul>
    <ul> 'days_exposition' (длительность размещения объявления (от публикации до снятия)); </ul><br>, а также: 
    <ul> 'floors_total' (количество этажей в доме); </ul> 
    <ul> 'airports_nearest' (расстояние до ближайшего аэропорта в метрах (м)); </ul> 
    <ul> 'parks_nearest' (расстояние до ближайшего парка); </ul>
    <ul> 'ponds_nearest' (расстояние до ближайшего пруда). </ul>          
    
<br>    
    Судя на третьей строчке проектной работы по установлению параметров и проведению предобработки и исследовательского анализа данных с архивными объявлениями о продаже квартир в г. Санкт-Петербург: мы обнаруживаем, что вместо представления данных в столбцах: 'studio' (наличие квартиры-студии) и 'open_plan'. <br>
<br>
     Также данные столбцов 'balcony', 'floors_total', а также 'days_exposition' приведём к типу данных 'int' (насчёт столбца 'days_exposition' - ни один SEO-специалист не будет ослеживать длительность с точностью до минуты, для удобства длительность размещения объявления будет измеряться в целых числах дней).<br>
</div>

In [13]:
data['is_apartment'] = data['is_apartment'].fillna('False')
display(data.head())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,False,...,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,False,...,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,False,...,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,False,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,False,...,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


In [14]:
data['balcony'] = (data['balcony'].fillna(0)).astype('int')
display(data.head())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,False,...,25.0,0,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,False,...,11.0,2,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,False,...,8.3,0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,False,...,NaN,0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,False,...,41.0,0,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


In [ ]:
data['locality_name'] = data['locality_name'].fillna('Санкт-Петербург')
#display(data.head())
display(data[1090:1110])

In [15]:
data['locality_name'] = data['locality_name'].fillna('Санкт-Петербург')
display(data.head())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,False,...,25.0,0,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,False,...,11.0,2,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,False,...,8.3,0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,False,...,NaN,0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,False,...,41.0,0,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


In [16]:
data_days_exposition_median = data['days_exposition'].median()
data_days_exposition_median

95.0

In [17]:
data['days_exposition'] = (data['days_exposition'].fillna(data_days_exposition_median)).astype('int')
display(data.head())

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,False,...,25.0,0,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,95
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,False,...,11.0,2,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,False,...,8.3,0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,False,...,NaN,0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,False,...,41.0,0,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121


In [18]:
data_floors_total_median = data['floors_total'].median()
data_floors_total_median

9.0

In [19]:
data['floors_total'] = (data['floors_total'].fillna(data_floors_total_median)).astype('int')
display(data.head(50))

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16,51.00,8,False,...,25.00,0,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,95
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11,18.60,1,False,...,11.00,2,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5,34.30,4,False,...,8.30,0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14,NaN,9,False,...,NaN,0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14,32.00,13,False,...,41.00,0,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12,14.40,5,False,...,9.10,0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26,10.60,6,False,...,14.40,1,посёлок Парголово,52996.0,19143.0,0.0,NaN,0.0,NaN,155
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24,NaN,22,False,...,18.90,2,Санкт-Петербург,23982.0,11634.0,0.0,NaN,0.0,NaN,95
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27,15.43,26,False,...,8.81,0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9,43.60,7,False,...,6.50,2,Санкт-Петербург,50898.0,15008.0,0.0,NaN,0.0,NaN,289


In [20]:
#data.query("locality_name != 'Санкт-Петербург' ")
#data.query('airports_nearest != airports_nearest')
#data[data['airports_nearest'].isna()]

data_unique = data['locality_name'].unique()
data_unique

array(['Санкт-Петербург', 'посёлок Шушары', 'городской посёлок Янино-1',
       'посёлок Парголово', 'посёлок Мурино', 'Ломоносов', 'Сертолово',
       'Петергоф', 'Пушкин', 'деревня Кудрово', 'Коммунар', 'Колпино',
       'поселок городского типа Красный Бор', 'Гатчина', 'поселок Мурино',
       'деревня Фёдоровское', 'Выборг', 'Кронштадт', 'Кировск',
       'деревня Новое Девяткино', 'посёлок Металлострой',
       'посёлок городского типа Лебяжье',
       'посёлок городского типа Сиверский', 'поселок Молодцово',
       'поселок городского типа Кузьмоловский',
       'садовое товарищество Новая Ропша', 'Павловск',
       'деревня Пикколово', 'Всеволожск', 'Волхов', 'Кингисепп',
       'Приозерск', 'Сестрорецк', 'деревня Куттузи', 'посёлок Аннино',
       'поселок городского типа Ефимовский', 'посёлок Плодовое',
       'деревня Заклинье', 'поселок Торковичи', 'поселок Первомайское',
       'Красное Село', 'посёлок Понтонный', 'Сясьстрой', 'деревня Старая',
       'деревня Лесколово', '

In [21]:
#data.query("locality_name != 'Санкт-Петербург' ")
#data_unique.to_csv('')
#data['locality_name'].isin('Кузьмоловский')

In [22]:
#if(data[data['parks_around3000'] == 0.0]):

#data_mean.loc[data_mean['total_income'] != data_mean['total_income'], 'total_income'] = data_mean_total_income_mean  
#display(data_mean.head(30))

  
#data['parks_around3000'] = (data['parks_around3000']).astype('int')
#if()
#data['parks_around3000'] = data[data['parks_around3000'] == 0.0]

#data.loc[:, 'parks_around3000'] = (data.loc[:, 'parks_around3000']).astype('int')
 

In [23]:
data.loc[~(data['parks_around3000'] != data['parks_around3000']), 'parks_nearest'] = (data.loc[:, 'parks_nearest'].fillna(0)).astype('int')

display(data.head(50)) 

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16,51.00,8,False,...,25.00,0,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,95
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11,18.60,1,False,...,11.00,2,посёлок Шушары,12817.0,18603.0,0.0,0.0,0.0,NaN,81
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5,34.30,4,False,...,8.30,0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14,NaN,9,False,...,NaN,0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14,32.00,13,False,...,41.00,0,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12,14.40,5,False,...,9.10,0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26,10.60,6,False,...,14.40,1,посёлок Парголово,52996.0,19143.0,0.0,0.0,0.0,NaN,155
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24,NaN,22,False,...,18.90,2,Санкт-Петербург,23982.0,11634.0,0.0,0.0,0.0,NaN,95
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27,15.43,26,False,...,8.81,0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9,43.60,7,False,...,6.50,2,Санкт-Петербург,50898.0,15008.0,0.0,0.0,0.0,NaN,289


In [24]:
#(data.query("ponds_around3000 == 0 ")).astype('int')
#data['ponds_around3000'] = (data['ponds_around3000']).astype('int')
#data.loc[:, 'ponds_around3000'] = (data.loc[:, 'ponds_around3000']).astype('int')

In [25]:
data.loc[~(data['ponds_around3000'] != data['ponds_around3000']), 'ponds_nearest'] = (data.loc[:, 'ponds_nearest'].fillna(0.0)).astype('int')

display(data.head(50))  

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.00,2019-03-07T00:00:00,3,2.70,16,51.00,8,False,...,25.00,0,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,95
1,7,3350000.0,40.40,2018-12-04T00:00:00,1,NaN,11,18.60,1,False,...,11.00,2,посёлок Шушары,12817.0,18603.0,0.0,0.0,0.0,0.0,81
2,10,5196000.0,56.00,2015-08-20T00:00:00,2,NaN,5,34.30,4,False,...,8.30,0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558
3,0,64900000.0,159.00,2015-07-24T00:00:00,3,NaN,14,NaN,9,False,...,NaN,0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424
4,2,10000000.0,100.00,2018-06-19T00:00:00,2,3.03,14,32.00,13,False,...,41.00,0,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121
5,10,2890000.0,30.40,2018-09-10T00:00:00,1,NaN,12,14.40,5,False,...,9.10,0,городской посёлок Янино-1,NaN,NaN,NaN,NaN,NaN,NaN,55
6,6,3700000.0,37.30,2017-11-02T00:00:00,1,NaN,26,10.60,6,False,...,14.40,1,посёлок Парголово,52996.0,19143.0,0.0,0.0,0.0,0.0,155
7,5,7915000.0,71.60,2019-04-18T00:00:00,2,NaN,24,NaN,22,False,...,18.90,2,Санкт-Петербург,23982.0,11634.0,0.0,0.0,0.0,0.0,95
8,20,2900000.0,33.16,2018-05-23T00:00:00,1,NaN,27,15.43,26,False,...,8.81,0,посёлок Мурино,NaN,NaN,NaN,NaN,NaN,NaN,189
9,18,5400000.0,61.00,2017-02-26T00:00:00,3,2.50,9,43.60,7,False,...,6.50,2,Санкт-Петербург,50898.0,15008.0,0.0,0.0,0.0,0.0,289


In [26]:
#for i in range(1, len(data['locality_name'].unique())):
#    print(data['locality_name'].unique()[i])

for value in data['locality_name'].unique():
    print(value)

Санкт-Петербург
посёлок Шушары
городской посёлок Янино-1
посёлок Парголово
посёлок Мурино
Ломоносов
Сертолово
Петергоф
Пушкин
деревня Кудрово
Коммунар
Колпино
поселок городского типа Красный Бор
Гатчина
поселок Мурино
деревня Фёдоровское
Выборг
Кронштадт
Кировск
деревня Новое Девяткино
посёлок Металлострой
посёлок городского типа Лебяжье
посёлок городского типа Сиверский
поселок Молодцово
поселок городского типа Кузьмоловский
садовое товарищество Новая Ропша
Павловск
деревня Пикколово
Всеволожск
Волхов
Кингисепп
Приозерск
Сестрорецк
деревня Куттузи
посёлок Аннино
поселок городского типа Ефимовский
посёлок Плодовое
деревня Заклинье
поселок Торковичи
поселок Первомайское
Красное Село
посёлок Понтонный
Сясьстрой
деревня Старая
деревня Лесколово
посёлок Новый Свет
Сланцы
село Путилово
Ивангород
Мурино
Шлиссельбург
Никольское
Зеленогорск
Сосновый Бор
поселок Новый Свет
деревня Оржицы
деревня Кальтино
Кудрово
поселок Романовка
посёлок Бугры
поселок Бугры
поселок городского типа Рощино
Кириши

In [27]:
'''cities = []
for value in data['locality_name'].unique():
    if(('городской посёлок' in value) | ('поселок' in value) | ('посёлок городского типа' in value) | 
       ('садовое товарищество' in value) | ('деревня Агалатово' in value) | ('деревня Чудской Бор' in value) | 
       ('посёлок Петро-Славянка' in value) | ('деревня Старое Хинколово' in value) | ('садоводческое некоммерческое товарищество' in value) | 
       ('посёлок Платформа 69-й километр' in value) | ('посёлок Кикерино' in value) | ('садоводческое некоммерческое товарищество Лесная Поляна' in value) |
       ('деревня Хязельки' in value) | ('посёлок Кикерино' in value)):
        continue
    else:
        print(value)
        cities.append(value)'''

"cities = []\nfor value in data['locality_name'].unique():\n    if(('городской посёлок' in value) | ('поселок' in value) | ('посёлок городского типа' in value) | \n       ('садовое товарищество' in value) | ('деревня Агалатово' in value) | ('деревня Чудской Бор' in value) | \n       ('посёлок Петро-Славянка' in value) | ('деревня Старое Хинколово' in value) | ('садоводческое некоммерческое товарищество' in value) | \n       ('посёлок Платформа 69-й километр' in value) | ('посёлок Кикерино' in value) | ('садоводческое некоммерческое товарищество Лесная Поляна' in value) |\n       ('деревня Хязельки' in value) | ('посёлок Кикерино' in value)):\n        continue\n    else:\n        print(value)\n        cities.append(value)"

In [28]:
'''airport_Pulkovo = (app.geocode('аэропорт Пулково').raw['lat'], app.geocode('аэропорт Пулково').raw['lon'])

for value in cities:
    point = (app.geocode(value).raw['lat'], app.geocode(value).raw['lon'])
    print(f'Рассстояние между двумя пунктами аэропорт Пулково и {value}: ', round(gd(airport_Pulkovo, point).m, 1))'''

"airport_Pulkovo = (app.geocode('аэропорт Пулково').raw['lat'], app.geocode('аэропорт Пулково').raw['lon'])\n\nfor value in cities:\n    point = (app.geocode(value).raw['lat'], app.geocode(value).raw['lon'])\n    print(f'Рассстояние между двумя пунктами аэропорт Пулково и {value}: ', round(gd(airport_Pulkovo, point).m, 1))"

In [29]:
cities = []
for value in data['locality_name'].unique():
    print(value)
    cities.append(value)

Санкт-Петербург
посёлок Шушары
городской посёлок Янино-1
посёлок Парголово
посёлок Мурино
Ломоносов
Сертолово
Петергоф
Пушкин
деревня Кудрово
Коммунар
Колпино
поселок городского типа Красный Бор
Гатчина
поселок Мурино
деревня Фёдоровское
Выборг
Кронштадт
Кировск
деревня Новое Девяткино
посёлок Металлострой
посёлок городского типа Лебяжье
посёлок городского типа Сиверский
поселок Молодцово
поселок городского типа Кузьмоловский
садовое товарищество Новая Ропша
Павловск
деревня Пикколово
Всеволожск
Волхов
Кингисепп
Приозерск
Сестрорецк
деревня Куттузи
посёлок Аннино
поселок городского типа Ефимовский
посёлок Плодовое
деревня Заклинье
поселок Торковичи
поселок Первомайское
Красное Село
посёлок Понтонный
Сясьстрой
деревня Старая
деревня Лесколово
посёлок Новый Свет
Сланцы
село Путилово
Ивангород
Мурино
Шлиссельбург
Никольское
Зеленогорск
Сосновый Бор
поселок Новый Свет
деревня Оржицы
деревня Кальтино
Кудрово
поселок Романовка
посёлок Бугры
поселок Бугры
поселок городского типа Рощино
Кириши

In [ ]:
unknown_points = []
app = nm(user_agent="tutorial")
airport_Pulkovo = (app.geocode('аэропорт Пулково').raw['lat'], app.geocode('аэропорт Пулково').raw['lon'])

for value in cities:
    try:
        point = (app.geocode(value).raw['lat'], app.geocode(value).raw['lon'])
        print(f'Расстояние между двумя пунктами: аэропорт Пулково и {value} -', round(gd(airport_Pulkovo, point).m, 1), 'метров')
    except:
        print(f'Населённый пункт {value} не найден!')
        unknown_points.append(value)

Расстояние между двумя пунктами: аэропорт Пулково и Санкт-Петербург - 14407.8 метров
Расстояние между двумя пунктами: аэропорт Пулково и посёлок Шушары - 6650.2 метров
Населённый пункт городской посёлок Янино-1 не найден!
Расстояние между двумя пунктами: аэропорт Пулково и посёлок Парголово - 30672.0 метров
Расстояние между двумя пунктами: аэропорт Пулково и посёлок Мурино - 4433098.4 метров
Расстояние между двумя пунктами: аэропорт Пулково и Ломоносов - 30081.0 метров
Расстояние между двумя пунктами: аэропорт Пулково и Сертолово - 38249.8 метров
Расстояние между двумя пунктами: аэропорт Пулково и Петергоф - 21843.3 метров
Расстояние между двумя пунктами: аэропорт Пулково и Пушкин - 12150.6 метров
Расстояние между двумя пунктами: аэропорт Пулково и деревня Кудрово - 20491.0 метров
Расстояние между двумя пунктами: аэропорт Пулково и Коммунар - 21186.8 метров
Расстояние между двумя пунктами: аэропорт Пулково и Колпино - 19833.4 метров
Населённый пункт поселок городского типа Красный Бор 

Расстояние между двумя пунктами: аэропорт Пулково и поселок Заводской - 659857.3 метров
Расстояние между двумя пунктами: аэропорт Пулково и городской посёлок Новоселье - 920401.7 метров
Расстояние между двумя пунктами: аэропорт Пулково и деревня Большие Колпаны - 32790.6 метров
Расстояние между двумя пунктами: аэропорт Пулково и деревня Горбунки - 15785.6 метров
Расстояние между двумя пунктами: аэропорт Пулково и деревня Батово - 57665.8 метров
Расстояние между двумя пунктами: аэропорт Пулково и деревня Заневка - 20491.0 метров
Расстояние между двумя пунктами: аэропорт Пулково и деревня Иссад - 119913.8 метров
Расстояние между двумя пунктами: аэропорт Пулково и Приморск - 111374.5 метров
Расстояние между двумя пунктами: аэропорт Пулково и городской посёлок Фёдоровское - 614993.4 метров
Расстояние между двумя пунктами: аэропорт Пулково и деревня Мистолово - 37278.0 метров
Расстояние между двумя пунктами: аэропорт Пулково и Новая Ладога - 119280.6 метров
Расстояние между двумя пунктами: 

In [ ]:
city_point
#display(data.head(10))
for item in city_point:
    data.loc[(data['airports_nearest'] != data['airports_nearest']) & (data['locality_name'] == item[0]), 'airports_nearest'] = item[1] 
    #data.loc[data['airports_nearest'] != data['airports_nearest'], 'airports_nearest'] = item[1] 
display(data.head(10))

'floors_total' (количество этажей в доме);
'airports_nearest' (расстояние до ближайшего аэропорта в метрах (м));
'parks_nearest' (расстояние до ближайшего парка);
'ponds_nearest'

### 0.2 Шаг 2.2 Приведение столбцов к нужным типам данных.

### 0.3 Шаг 3.1 Подсчёт данных и добавление в новые столбцы.

### 0.4 Шаг 4.1 Проведите исследовательский анализ данных.

## 1 Общий вывод: